In [1]:
pip install opencv-python mediapipe


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\rihem\appdata\local\programs\python\python311\lib\site-packages\deepface-0.0.86-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# modules MediaPipe 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

#  capture vidéo
cap = cv2.VideoCapture(0)

#  distance entre deux points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        
      
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Traiter l'image pour la détection des mains
        results = hands.process(image_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Obtenir les coordonnées des points de repère
                landmarks = hand_landmarks.landmark
                
                # Calculer la distance entre le poignet et l'index
                wrist = np.array([landmarks[0].x, landmarks[0].y])
                index_finger_tip = np.array([landmarks[8].x, landmarks[8].y])
                distance = calculate_distance(wrist, index_finger_tip)
                
                # Convertir les coordonnées des points de repère en pixels
                wrist_pixel = (int(wrist[0] * frame.shape[1]), int(wrist[1] * frame.shape[0]))
                index_finger_tip_pixel = (int(index_finger_tip[0] * frame.shape[1]), int(index_finger_tip[1] * frame.shape[0]))
                
                # Dessiner un rectangle autour de la main
                x_min = min(wrist_pixel[0], index_finger_tip_pixel[0])
                y_min = min(wrist_pixel[1], index_finger_tip_pixel[1])
                x_max = max(wrist_pixel[0], index_finger_tip_pixel[0])
                y_max = max(wrist_pixel[1], index_finger_tip_pixel[1])
                cv2.rectangle(frame, (x_min - 20, y_min - 20), (x_max + 20, y_max + 20), (0, 0, 255), 2)
                
                # Afficher la distance calculée sur le flux vidéo
                distance_text = f'Distance: {distance:.2f}'
                cv2.putText(frame, distance_text, (x_min, y_min - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                # Dessiner les landmarks de la main
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Afficher le flux vidéo
        cv2.imshow('Hand Detection', frame)
        
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
